In [124]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [125]:
Predicted = pd.read_csv("./Stereoscope/Outputs/output_Sim13.csv")
Target=pd.read_csv("./Data/Sim13/target.csv")

In [126]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)
#Predicted.drop(columns="prediction.score.max",inplace=True)


In [127]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [128]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [129]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [130]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [131]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [132]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [133]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [134]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [135]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [136]:
np.mean(np.nan_to_num(d))

0.7443041846913265

In [137]:
d

array([0.92527076, 0.77893777, 0.73268157, 0.48142485, 0.19641927,
       0.89730725, 0.89544373, 0.91800044, 0.87922614, 0.73833008])

## Jenson Shannon Divergence

In [138]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.43882365499486087

## F1 Score, Precision, Recall

In [139]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [140]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [141]:
d

,Precision,Recall,F1-score
0.100,0.580957,0.816661,0.678934
0.200,0.511986,0.679237,0.583870
0.300,0.496970,0.156988,0.238603
0.010,0.534884,0.926378,0.678187
0.001,0.457876,0.936613,0.615068


## Wasserstein Distance

In [142]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.05349856656358589

## KL Divergence

In [143]:
to_be_kept=(Predicted>0)

In [144]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [145]:
kl=scipy.special.rel_entr(Target,Predicted)

In [146]:
np.mean(kl)

0.09417751286245984